In [21]:
import pandas as pd
import json
from pathlib import Path

In [22]:
def read_jsonl_file(file_path):
    """
    JSON Lines 형식의 파일을 읽는 함수
    """
    data = []
    try:
        if not Path(file_path).is_file():
            print(f"파일이 존재하지 않습니다: {file_path}")
            return None

        with open(file_path, 'r', encoding='utf-8') as f:
            for line_number, line in enumerate(f, start=1):
                try:
                    json_obj = json.loads(line.strip())
                    data.append(json_obj)
                except json.JSONDecodeError as e:
                    print(f"JSON 파싱 오류 (라인 {line_number}, 무시됨): {str(e)}")
                    continue
        return data
    except Exception as e:
        print(f"파일 읽기 오류 ({file_path}): {str(e)}")
        return None

def compare_non_matching_rows(reference_path, comparison_paths):
    """
    100% 일치하지 않는 행들만 비교하는 함수
    eval_id와 standalone_query 포함
    """
    reference_data = read_jsonl_file(reference_path)
    if not reference_data:
        print("기준 파일을 읽을 수 없습니다.")
        return None

    reference_name = Path(reference_path).stem
    comparison_results = []

    for comp_path in comparison_paths:
        comp_data = read_jsonl_file(comp_path)
        if not comp_data:
            continue

        comp_name = Path(comp_path).stem
        
        # 각 행별로 비교
        for idx, (ref_item, comp_item) in enumerate(zip(reference_data, comp_data)):
            ref_topk = set(ref_item['topk'])
            comp_topk = set(comp_item['topk'])
            
            # 100% 일치하지 않는 경우만 처리
            if ref_topk != comp_topk:
                intersection = ref_topk & comp_topk
                ref_unique = ref_topk - comp_topk
                comp_unique = comp_topk - ref_topk
                
                comparison_results.append({
                    'Row': idx + 1,
                    'Eval ID': ref_item.get('eval_id', ''),
                    'Standalone Query': ref_item.get('standalone_query', ''),
                    'Reference File': reference_name,
                    'Comparison File': comp_name,
                    'Common Values': sorted(list(intersection)),
                    'Only in Reference': sorted(list(ref_unique)),
                    'Only in Comparison': sorted(list(comp_unique)),
                    'Match Count': len(intersection),
                    'Similarity': f"{(len(intersection) / 3) * 100:.2f}%"
                })

    return comparison_results

def save_detailed_results(results, output_file):
    """
    상세 비교 결과를 CSV 파일로 저장하고 콘솔에 출력
    """
    if not results:
        print("비교할 결과가 없습니다.")
        return

    df = pd.DataFrame(results)
    
    # 결과를 일치 개수의 내림차순으로 정렬
    df = df.sort_values(by=['Match Count', 'Row'], ascending=[False, True])
    
    df.to_csv(output_file, index=False)
    print(f"\n불일치 행에 대한 상세 비교 결과가 저장됨: {output_file}")
    
    # 콘솔에 상세 정보 출력
    print(f"\n총 {len(results)}개의 불일치 행이 발견됨")
    print("\n=== 상세 비교 결과 ===")
    
    for result in results:
        print(f"\n[행 {result['Row']}] 일치율: {result['Similarity']}")
        print(f"Eval ID: {result['Eval ID']}")
        print(f"Standalone Query: {result['Standalone Query']}")
        print(f"공통 값: {result['Common Values']}")
        print(f"기준 파일에만 있는 값: {result['Only in Reference']}")
        print(f"비교 파일에만 있는 값: {result['Only in Comparison']}")
        print("-" * 80)

In [23]:
def main():
    # 파일 경로 설정
    reference_file = '/home/0.8856.csv'
    comparison_files = [        
        '/home/qurey수정_final.csv'
    ]

    # 불일치 행 비교 수행
    results = compare_non_matching_rows(reference_file, comparison_files)
    
    if results:
        # 결과 저장 및 출력
        output_file = '/home/IR/data/detailed_comparison_results.csv'
        save_detailed_results(results, output_file)
        
        # 일치율 분포 출력
        match_counts = pd.DataFrame(results)['Match Count'].value_counts()
        print("\n[불일치 행들의 일치 개수 분포]")
        for count, freq in match_counts.items():
            print(f" - {count}개 일치: {freq}개 행")
    else:
        print("비교 수행 중 오류가 발생했습니다.")

if __name__ == "__main__":
    main()


불일치 행에 대한 상세 비교 결과가 저장됨: /home/IR/data/detailed_comparison_results.csv

총 16개의 불일치 행이 발견됨

=== 상세 비교 결과 ===

[행 5] 일치율: 33.33%
Eval ID: 280
Standalone Query: Dmitri Ivanovsky가 누구야?
공통 값: ['38686456-b993-4cbb-af0d-1c53df2f3e12']
기준 파일에만 있는 값: ['20da1c0b-8ba9-425b-a53c-014391560737', 'f5034f05-4a40-4f7e-b011-ea08ba869c7c']
비교 파일에만 있는 값: ['0d2d467d-65b3-46af-b7c2-c931708706f6', '5adedff9-66cd-4822-ba07-dc8735dc8d8c']
--------------------------------------------------------------------------------

[행 9] 일치율: 33.33%
Eval ID: 42
Standalone Query: 이란 콘트라 사건이 뭐야 이란-콘트라 사건은 로널드 레이건 집권기인 1986년에 레이건 행정부와 CIA가 적성국이었던 이란에게 무기를 몰래 수출한 대금으로 니카라과의 우익 성향 반군 콘트라를 지원하면서 동시에 반군으로부터 마약을 사들인 후 미국에 판매하다가 발각되어 큰 파장을 일으킨 사건입니다. 이 사건이 미국 정치에 미친 영향은?
공통 값: ['4b49f3a2-32c9-4b2e-89c4-4719f98e7a74']
기준 파일에만 있는 값: ['2fe9c75a-e8e9-4bff-a87c-6d1f3289fe55', 'f7d4f91c-49ff-44cd-9f81-c6bef6ad2d37']
비교 파일에만 있는 값: ['2e91a956-a24a-4d0c-8a9a-7aeadba95ad9', 'e181b15e-d1a2-445e-8867-2bc0b6a0ae65']
---------------------------